<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "input"
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

### Download Data

We will use [Walmart annual report PDFs](https://github.com/sujee/data/tree/main/data-prep-kit/walmart-reports-1) as our input data.

Feel free to substitute your data

In [2]:
import os, sys
import shutil
from utils import download_file

## Download the data files
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart-10K-Reports-Optimized_2023.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart-10K-Reports-Optimized_2023.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024_copy.pdf' ))  # create a dupe file

Local file 'input/Walmart-10K-Reports-Optimized_2023.pdf' (1.61 MB) already exists. Skipping download.
Local file 'input/Walmart_2024.pdf' (4.87 MB) already exists. Skipping download.
Local file 'input/Walmart_2024_copy.pdf' (4.87 MB) already exists. Skipping download.


### Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### Import Common python modules

In [4]:
import os
import sys

# Main repo root
from utils import rootdir

from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

STAGE = 0

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### Set Input/output Folder

In [5]:
STAGE  += 1
# STAGE = 1  ## DEBUG

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_parquet_out")

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'


### Execute 

In [6]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


10:27:06 INFO - Running locally
10:27:06 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': False}
10:27:06 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
10:27:06 INFO - data factory data_ max_files -1, n_sample -1
10:27:06 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
10:27:06 INFO - pipeline id pipeline_id
10:27:06 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
10:27:06 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
10:27:06 INFO - actor creation delay 0
10:27:06 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
2024-08-30 10:27:08,526	I

✅ Stage:1 completed successfully
CPU times: user 4.08 s, sys: 1.07 s, total: 5.15 s
Wall time: 10min 48s


### Inspect Generated output

Here we should see one entry per input file processed

In [7]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (3, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,Walmart-10K-Reports-Optimized_2023.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,Walmart-10K-Reports-Optimized_2023.pdf
1,Walmart_2024_copy.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,82,1163,95cc2911-9a0d-49c3-a259-c74e35fca3ea,pdf,0be5657667eb7229f1389625f61b6d6dfb608c617ed5ef...,1112050,2024-08-30T10:37:40.616022,299.935132,Walmart_2024_copy.pdf
2,Walmart_2024.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,82,1163,00df8499-2863-4ca4-96dc-0c2a2014c3dc,pdf,dd3b262828146a536bdc0f04e7c9dfbd7406d043714989...,1112045,2024-08-30T10:32:40.640835,312.142404,Walmart_2024.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### Set Input/output Folder

In [8]:
STAGE  += 1
# STAGE = 2  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_chunk_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### Execute 

In [9]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/sujee/apps/anaconda3/envs/data-prep-
[nltk_data]     kit-3-py311/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!
10:37:53 INFO - Running locally
10:37:53 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
10:37:53 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
10:37:53 INFO - data factory data_ max_files -1, n_sample -1
10:37:53 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:37:53 INFO - pipeline id pi

✅ Stage:2 completed successfully
CPU times: user 1.35 s, sys: 997 ms, total: 2.35 s
Wall time: 18.5 s


### Inspect Generated output

We would see documents are split into many chunks

In [10]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(3)

Files processed : 3
Chunks created : 2,042
Input data dimensions (rows x columns)=  (3, 12)
Output data dimensions (rows x columns)=  (2042, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
1229,Walmart_2024_copy.pdf,100,82,1163,95cc2911-9a0d-49c3-a259-c74e35fca3ea,pdf,0be5657667eb7229f1389625f61b6d6dfb608c617ed5ef...,1112050,2024-08-30T10:37:40.616022,299.935132,Walmart_2024_copy.pdf,"#26*1.88*) &62.2,7\n*F=CF HC H<9 .5L IHG 5B8...",$.main-text[891],76,"[35.41, 538.52, 546.86, 609.18]"
1767,Walmart_2024.pdf,100,82,1163,00df8499-2863-4ca4-96dc-0c2a2014c3dc,pdf,dd3b262828146a536bdc0f04e7c9dfbd7406d043714989...,1112045,2024-08-30T10:32:40.640835,312.142404,Walmart_2024.pdf,67:?:E:@? 2?5 #:>:E2E:@?D @7 ?E6C?2= @?EC@=...,$.main-text[630],55,"[35.55, 222.69, 525.53, 256.91]"
865,Walmart_2024_copy.pdf,100,82,1163,95cc2911-9a0d-49c3-a259-c74e35fca3ea,pdf,0be5657667eb7229f1389625f61b6d6dfb608c617ed5ef...,1112050,2024-08-30T10:37:40.616022,299.935132,Walmart_2024_copy.pdf,.6 C6=J 6IE6?D:G6=J @? :?7@C>2E:@? 2?5 7:?2?4:...,$.main-text[278],25,"[35.23, 641.07, 547.64, 747.74]"


## Step-4: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### Set Input/output Folder

In [11]:
STAGE  += 1
# STAGE  = 3  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_ededupe_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_ededupe_out'


### Execute 

In [12]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

10:38:10 INFO - Running locally
10:38:10 INFO - exact dedup params are {'doc_column': 'contents', 'hash_cpu': 0.5, 'num_hashes': 2}
10:38:10 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_ededupe_out
10:38:10 INFO - data factory data_ max_files -1, n_sample -1
10:38:10 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:38:10 INFO - pipeline id pipeline_id
10:38:10 INFO - code location None
10:38:10 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
10:38:10 INFO - actor creation delay 0
10:38:10 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
2024-08-30 10:38:12,554	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=1090364) 10:38:13 INFO - orchestrator 

✅ Stage:3 completed successfully
CPU times: user 121 ms, sys: 184 ms, total: 305 ms
Wall time: 14.8 s


### Inspect Generated output

In [13]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(3)

Input data dimensions (rows x columns)=  (2042, 15)
Output data dimensions (rows x columns)=  (1324, 15)
Input chunks before exact dedupe : 2,042
Output chunks after exact dedupe : 1,324
Duplicate chunks removed :   718


## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

In [14]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  += 1
# STAGE  = 4  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_doc_id_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_ededupe_out' --> output='output/04_doc_id_out'


In [15]:
%%time 

from doc_id_transform_ray import DocIDRayTransformConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

10:38:25 INFO - Running locally
10:38:25 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column'}
10:38:25 INFO - data factory data_ is using local data access: input_folder - output/03_ededupe_out output_folder - output/04_doc_id_out
10:38:25 INFO - data factory data_ max_files -1, n_sample -1
10:38:25 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:38:25 INFO - pipeline id pipeline_id
10:38:25 INFO - code location None
10:38:25 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
10:38:25 INFO - actor creation delay 0
10:38:25 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
2024-08-30 10:38:27,443	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=1092147) 

✅ Stage:4 completed successfully
CPU times: user 136 ms, sys: 159 ms, total: 296 ms
Wall time: 15 s


### Inspect Generated output

In [16]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(3)

Input data dimensions (rows x columns)=  (1324, 15)
Output data dimensions (rows x columns)=  (1324, 17)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,hash_column,int_id_column
860,Walmart_2024.pdf,100,82,1163,00df8499-2863-4ca4-96dc-0c2a2014c3dc,pdf,dd3b262828146a536bdc0f04e7c9dfbd7406d043714989...,1112045,2024-08-30T10:32:40.640835,312.142404,Walmart_2024.pdf,#682= +2I )68F=2E@CJ @>A=:2?46 )6AFE2E:@?...,$.main-text[299],27,"[35.24, 725.11, 503.48, 747.51]",97e06840b409f4ca176c2d5b145e8f25c9d3d37c6510ac...,187
2,Walmart-10K-Reports-Optimized_2023.pdf,100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,Walmart-10K-Reports-Optimized_2023.pdf,A A message from our r CEO\nFor the fiscal y...,$.main-text[14],3,"[214.16, 607.24, 390.6, 617.44]",2f26fa255117cd004e3fc8e4348d39fd265e570edfdbc7...,653
607,Walmart-10K-Reports-Optimized_2023.pdf,100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,Walmart-10K-Reports-Optimized_2023.pdf,"ITEM 15. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",$.main-text[978],85,"[111.85, 578.01, 263.73, 587.0]",43503987ec97f0b553f02f3572b2326006b641745b7c2f...,1258


## Step-6: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### Set Input/output Folder

In [17]:
## Input to this component is the output of doc_id generator component. 

STAGE  += 1
# STAGE  = 5  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_fdedupe_out")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_doc_id_out' --> output='output/05_fdedupe_out'


### Execute 

In [18]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

10:38:40 INFO - Running locally
10:38:40 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.5, 'mhash_cpu': 0.5, 'doc_cpu': 0.5, 'num_doc_actors': 2, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 2, 'num_permutations': 64, 'threshold': 0.8, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
10:38:40 INFO - data factory data_ is using local data access: input_folder - output/04_doc_id_out output_folder - output/05_fdedupe_out
10:38:40 INFO - data factory data_ max_files -1, n_sample -1
10:38:40 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:38:40 INFO - pipeline id pipeline_id
10:38:40 INFO - code location None
10:38:40 INFO - number of wor

✅ Stage:5 completed successfully
CPU times: user 208 ms, sys: 195 ms, total: 402 ms
Wall time: 37 s


### Inspect Generated output

In [19]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(3)

Input data dimensions (rows x columns)=  (1324, 15)
Output data dimensions (rows x columns)=  (1302, 17)
Duplicate chunks removed  by fuzzy-dedupe:   22


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,int_id_column,hash_column
1102,Walmart_2024.pdf,100,82,1163,00df8499-2863-4ca4-96dc-0c2a2014c3dc,pdf,dd3b262828146a536bdc0f04e7c9dfbd7406d043714989...,1112045,2024-08-30T10:32:40.640835,312.142404,Walmart_2024.pdf,"& % (' ('-+(%%#'! '- + ,-\n(CB7CBHFC@...",$.main-text[734],66,"[35.27, 647.11, 551.24, 729.93]",447,-1
470,Walmart-10K-Reports-Optimized_2023.pdf,100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,Walmart-10K-Reports-Optimized_2023.pdf,Share-Based Compensation\nFair value of restri...,$.tables[39],68,"[47.21, 61.57, 540.3, 152.39]",1123,-1
339,Walmart-10K-Reports-Optimized_2023.pdf,100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,Walmart-10K-Reports-Optimized_2023.pdf,Capital Resources\nWe believe our cash flows ...,$.main-text[517],48,"[46.39, 510.11, 539.0, 555.63]",992,-1


## Step-7: Document Quality

### Set Input/output Folder

In [20]:
STAGE  += 1
# STAGE  = 6 ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_doc_quality_out")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fdedupe_out' --> output='output/06_doc_quality_out'


### Execute 

In [21]:
%%time

import os
import sys
from pathlib import Path

from doc_quality_transform import (
    text_lang_cli_param,
    doc_content_column_cli_param,
    bad_word_filepath_cli_param,
)
from doc_quality_transform_ray import DocQualityRayTransformConfiguration
from data_processing.utils import ParamsUtils

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

doc_quality_basedir = os.path.join(rootdir, "transforms", "language", "doc_quality", "python")
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_creation_delay": 0,
    # doc quality configuration
    text_lang_cli_param: "en",
    doc_content_column_cli_param: "contents",
    bad_word_filepath_cli_param: os.path.join(doc_quality_basedir, "ldnoobw", "en"),
}


Path(output_folder).mkdir(parents=True, exist_ok=True)

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocQualityRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

10:39:17 INFO - Running locally
10:39:17 INFO - doc_quality parameters are : {'text_lang': 'en', 'doc_content_column': 'contents', 'bad_word_filepath': '/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee/transforms/language/doc_quality/python/ldnoobw/en', 's3_cred': None, 'docq_data_factory': <data_processing.data_access.data_access_factory.DataAccessFactory object at 0x79b73a1f9210>}
10:39:17 INFO - data factory docq_ is using local configuration without input/output path
10:39:17 INFO - data factory docq_ max_files -1, n_sample -1
10:39:17 INFO - data factory docq_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:39:17 INFO - data factory data_ is using local data access: input_folder - output/05_fdedupe_out output_folder - output/06_doc_quality_out
10:39:17 INFO - data factory data_ max_files -1, n_sample -1
10:39:17 INFO - data factory data_ Not using data sets, checkpointing False, 

✅ Stage:6 completed successfully
CPU times: user 139 ms, sys: 177 ms, total: 316 ms
Wall time: 15.6 s


### Inspect Generated output

In [22]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(3)

Input data dimensions (rows x columns)=  (1324, 15)
Output data dimensions (rows x columns)=  (1302, 28)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_mean_word_len,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words
354,Walmart-10K-Reports-Optimized_2023.pdf,100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,...,6.272727,0.0,2,0.0,0.0,False,0.0,0.0,1.000000,False
1125,Walmart_2024.pdf,100,82,1163,00df8499-2863-4ca4-96dc-0c2a2014c3dc,pdf,dd3b262828146a536bdc0f04e7c9dfbd7406d043714989...,1112045,2024-08-30T10:32:40.640835,312.142404,...,5.121622,0.0,31,0.0,0.0,False,0.0,0.0,0.648649,False
204,Walmart-10K-Reports-Optimized_2023.pdf,100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,...,5.880000,0.0,1,0.0,0.0,False,0.0,0.0,1.000000,True


## Step-8:   Text encoding

Encode text for the vector storage.

In [23]:
STAGE  += 1
# STAGE  = 7 ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_encoder_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-7: Processing input='output/06_doc_quality_out' --> output='output/07_encoder_out'


In [24]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

10:39:33 INFO - Running locally
10:39:33 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
10:39:33 INFO - data factory data_ is using local data access: input_folder - output/06_doc_quality_out output_folder - output/07_encoder_out
10:39:33 INFO - data factory data_ max_files -1, n_sample -1
10:39:33 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
10:39:33 INFO - pipeline id pipeline_id
10:39:33 INFO - code location None
10:39:33 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
10:39:33 INFO - actor creation delay 0
10:39:33 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
2024-08-30 10:39:35,588	INFO worker.py:1744 -- Started a local Ray instance. Vie

✅ Stage:7 completed successfully
CPU times: user 510 ms, sys: 285 ms, total: 795 ms
Wall time: 1min 4s


### Inspect Generated output

In [25]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(3)

Input data dimensions (rows x columns)=  (1302, 28)
Output data dimensions (rows x columns)=  (1302, 29)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words,embeddings
916,Walmart_2024.pdf,100,82,1163,00df8499-2863-4ca4-96dc-0c2a2014c3dc,pdf,dd3b262828146a536bdc0f04e7c9dfbd7406d043714989...,1112045,2024-08-30T10:32:40.640835,312.142404,...,0.00000,3,0.0,0.0,False,0.0,0.0,0.978022,False,"[-0.048175987, 0.0011802563, -0.046808466, -0...."
286,Walmart-10K-Reports-Optimized_2023.pdf,100,81,1163,a8118ae6-e6b5-4595-86ed-bf519ec23551,pdf,ea5544f26fe0831ec9befbf7aaf68b1b256df6c3ae18b8...,1159974,2024-08-30T10:32:49.798524,321.107279,...,0.00000,29,0.0,0.0,False,0.0,0.0,0.919414,True,"[0.0038028236, -0.13894859, 0.015160485, -0.00..."
852,Walmart_2024.pdf,100,82,1163,00df8499-2863-4ca4-96dc-0c2a2014c3dc,pdf,dd3b262828146a536bdc0f04e7c9dfbd7406d043714989...,1112045,2024-08-30T10:32:40.640835,312.142404,...,0.01087,4,0.0,0.0,False,0.0,0.0,0.978261,False,"[-0.033763092, 0.031698707, -0.04227217, 0.008..."


## Step-9: Copy output to final output dir

In [26]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/07_encoder_out' --> 'output/output_final'
